In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('Churn_Modelling.csv')

In [3]:
X = data.drop(columns=['Exited', 'CustomerId', 'Surname', 'RowNumber'])  # Exclude columns
y = data['Exited']  # Target

In [4]:
data.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [6]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [8]:
data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [9]:
# Step 3: Data Preprocessing
# Handle missing values and encode categorical variables

# Removing rows with missing values:
data = data.drop(['CustomerId', 'Surname', 'RowNumber'], axis = 1)
print(data.columns)

# Replacing missing values with a specific value (e.g., mean):
# data['column_name'].fillna(data['column_name'].mean(), inplace=True)

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')


In [10]:
# You need to ensure that the columns 'Geography' and 'Gender' are present in the DataFrame X
# Add additional error handling to verify the column names
columns_to_encode = ['Geography', 'Gender']
for column in columns_to_encode:
    if column not in X.columns:
        raise ValueError(f"Column '{column}' not found in the DataFrame X.")

# You need to encode categorical variables like "Geography" and "Gender" into numerical format using one-hot encoding.
X = pd.get_dummies(X, columns=['Geography', 'Gender'], drop_first=True)

In [11]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [12]:
# Step 5: Initialize and Build the Model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [13]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)

Epoch 1/20
250/250 [==============================] - 3s 4ms/step - loss: 0.4865 - accuracy: 0.7895
Epoch 2/20
250/250 [==============================] - 1s 4ms/step - loss: 0.4446 - accuracy: 0.8109
Epoch 3/20
250/250 [==============================] - 1s 4ms/step - loss: 0.4245 - accuracy: 0.8189
Epoch 4/20
250/250 [==============================] - 1s 4ms/step - loss: 0.4042 - accuracy: 0.8267
Epoch 5/20
250/250 [==============================] - 1s 4ms/step - loss: 0.3828 - accuracy: 0.8385
Epoch 6/20
250/250 [==============================] - 1s 4ms/step - loss: 0.3701 - accuracy: 0.8455
Epoch 7/20
250/250 [==============================] - 1s 3ms/step - loss: 0.3605 - accuracy: 0.8531
Epoch 8/20
250/250 [==============================] - 1s 3ms/step - loss: 0.3551 - accuracy: 0.8511
Epoch 9/20
250/250 [==============================] - 1s 3ms/step - loss: 0.3513 - accuracy: 0.8518
Epoch 10/20
250/250 [==============================] - 1s 3ms/step - loss: 0.3489 - accuracy: 0.8568

In [16]:
# Step 6: Evaluate the Model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)  # Convert to binary prediction

63/63 [==============================] - 0s 2ms/step


In [17]:
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(confusion)

Accuracy: 0.862
Confusion Matrix:
[[1563   44]
 [ 232  161]]
